# Laboratorio 5

# Análisis de Sentimientos

### Cristina Bautista 161260
### Andy Castillo 18040
### Marco Fuentes 18188
### Abril Palencia 18198

In [1]:
import numpy as np
import pandas as pd

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import FreqDist
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cristinabautista/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cristinabautista/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from wordcloud import WordCloud, STOPWORDS

import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
%matplotlib inline

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('GrammarandProductReviews.csv')

In [ ]:
data.info()

In [ ]:
data = data[['brand','categories','manufacturer', 'name', 'reviews.didPurchase', 'reviews.doRecommend', 'reviews.numHelpful', 'reviews.rating', 'reviews.text', 'reviews.title', 'reviews.username']]

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data['brand'] = data['brand'].str.lower()
data['categories'] = data['categories'].str.lower()
data['manufacturer'] = data['manufacturer'].str.lower()
data['name'] = data['name'].str.lower()
data['reviews.text'] = data['reviews.text'].str.lower()
data['reviews.title'] = data['reviews.title'].str.lower()
data['reviews.username'] = data['reviews.username'].str.lower()

In [ ]:
# Eliminar url
data['brand'] = data['brand'].str.replace(r'http\S+', ' ')
data['categories'] = data['categories'].str.replace(r'http\S+', ' ')
data['manufacturer'] = data['manufacturer'].str.replace(r'http\S+', ' ')
data['name'] = data['name'].str.replace(r'http\S+', ' ')
data['reviews.text'] = data['reviews.text'].str.replace(r'http\S+', ' ')
data['reviews.title'] = data['reviews.title'].str.replace(r'http\S+', ' ')
data['reviews.username'] = data['reviews.username'].str.replace(r'http\S+', ' ')

In [ ]:
# Se elimino todo lo que no sean letras, como emojis, signos de puntuacion y numeros
data['brand'] = data['brand'].str.replace(r'[^A-Za-z\s]+', ' ')
data['categories'] = data['categories'].str.replace(r'[^A-Za-z]+', ' ')
data['manufacturer'] = data['manufacturer'].str.replace(r'[^A-Za-z]+', ' ')
data['name'] = data['name'].str.replace(r'[^A-Za-z\s]+', ' ')
data['reviews.text'] = data['reviews.text'].str.replace(r'[^A-Za-z]+', ' ')
data['reviews.title'] = data['reviews.title'].str.replace(r'[^A-Za-z]+', ' ')
data['reviews.username'] = data['reviews.username'].str.replace(r'[^A-Za-z]+', ' ')

In [ ]:
# Se revisa que no tengan caracteres especiales las columnas deseadas
data.head()

In [ ]:
# Tokenize
tokenizer = RegexpTokenizer(r'\w+')

data['brand'] = data['brand'].apply(lambda x: tokenizer.tokenize(x))
data['categories'] = data['categories'].apply(lambda x: tokenizer.tokenize(x))
data['manufacturer'] = data['manufacturer'].apply(lambda x: tokenizer.tokenize(str(x)))
data['name'] = data['name'].apply(lambda x: tokenizer.tokenize(x))
data['reviews.text'] = data['reviews.text'].apply(lambda x: tokenizer.tokenize(str(x)))
data['reviews.title'] = data['reviews.title'].apply(lambda x: tokenizer.tokenize(str(x)))
data['reviews.username'] = data['reviews.username'].apply(lambda x: tokenizer.tokenize(str(x)))

In [ ]:
# Stopwords
stopwords = stopwords.words('english')

In [ ]:
data['reviews.text'] = data['reviews.text'].apply(lambda y: [w for w in y if w not in stopwords])
data['reviews.title'] = data['reviews.title'].apply(lambda y: [w for w in y if w not in stopwords])

In [ ]:
# Se revisa que en reviews text y title no contengan stopwords
data.head()